In [9]:
import os
import random
import shutil

from paths import src_dir

In [10]:
def add_suffix_to_path(path, suffix): 
    # Check if the path already ends with a slash
    if path.endswith('/'):
        new_path = path.rstrip('/') + suffix + '/'
    else:
        new_path = path + suffix + '/'
    return new_path

In [11]:
def random_split(parent_dir, ratio_train=0.8):
    """
    Randomly separate subdirectories within a parent directory and move them to
    separate output directories according to the given ratios.

    Parameters:
    - parent_dir: The parent directory containing subdirectories to be split.
    - ratios: A list of ratios (e.g., [0.7, 0.3]) specifying the split ratio.
    - output_dirs: A list of output directory names for the separated data.

    Returns:
    - None
    """
    # define the suffixes of the different data chunks
    suffixes = ["_train", "_validate"]

    # adding suffixes to form output directories
    output_dirs = [add_suffix_to_path(parent_dir, suffix) for suffix in suffixes]

    # count the subdirectories that will be moved
    subdirectories = [d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]

    # Create output directories if they don't exist
    for output_dir in output_dirs:
        os.makedirs(output_dir, exist_ok=True)

    random.shuffle(subdirectories)

    # calculate the position of split
    total_dirs = len(subdirectories)
    split_point = int(ratio_train * total_dirs)

    train_dirs = subdirectories[:split_point]
    test_dirs = subdirectories[split_point:]
    both_dirs = [train_dirs, test_dirs]

    for move_idx in range(len(both_dirs)): 
        move_dirs = both_dirs[move_idx]
        output_dir = output_dirs[move_idx]

        for move_dir in move_dirs: 
            source = os.path.join(parent_dir, move_dir)
            destination = os.path.join(output_dir, move_dir)
            shutil.move(source, destination)


if __name__ == '__main__':
    random_split(os.path.join(src_dir, "try/one/"), 0.8)

In [1]:
import torch
import torch.nn as nn

In [23]:
ks = 3
x = torch.rand((10, 21, 3))
conv = nn.Conv1d(in_channels=3, out_channels=5, kernel_size=ks)
act = nn.ReLU()
mp = nn.MaxPool1d(kernel_size=21 - ks + 1)

In [24]:
y = conv(x.permute(0, 2, 1))
y = act(y)
y = mp(y)

In [25]:
y = y.permute(0, 2, 1)

In [26]:
y.shape

torch.Size([10, 1, 5])

In [3]:
import fontTools.ttLib as tt
from paths import *

In [5]:
def list_font_characters(font_path):
    font = tt.TTFont(font_path)
    cmap = font.getBestCmap()
    # characters = sorted(cmap.keys())
    characters = [chr(char_code) for char_code in sorted(cmap.keys())]
    return characters

font_path = src_dir + "handshape.TTF"
characters = list_font_characters(font_path)
print(characters)

[' ', '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '‐']


In [6]:
len(characters)

95

In [7]:
import pandas as pd

In [45]:
guideline = pd.read_csv(guide_path)

In [46]:
all_list = []

for hs in ['DH_1', 'OH_1', 'DH_2', 'OH_2', 'DH_3', 'OH_3', 'DH_4', 'OH_4']: 
    all_list += guideline[(guideline[hs].notna())&(guideline[hs] != "NONE")][hs].tolist()

In [49]:
str(sorted(set(all_list)))

"[')', ',', '-', '0', '1', '2', '3', '4', '5', '6', '8', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '\\\\', 'b', 'd', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}']"

In [1]:
import os
import svgwrite
from PIL import Image, ImageDraw, ImageFont

In [2]:
def generate_svg(font_path, output_path):
    # Define the characters you want to include in the SVG
    characters = [chr(i) for i in range(32, 127)]  # ASCII characters

    # Load the font
    font_size = 30
    font = ImageFont.truetype(font_path, font_size)

    # Determine the size of the SVG canvas
    canvas_width = font_size * len(characters)
    canvas_height = font_size * 2

    # Create an SVG drawing
    dwg = svgwrite.Drawing(output_path, profile='tiny', size=(f"{canvas_width}px", f"{canvas_height}px"))

    # Draw each character on the SVG canvas
    for i, char in enumerate(characters):
        x_position = i * font_size
        y_position = font_size

        dwg.add(dwg.text(char, insert=(f"{x_position}px", f"{y_position}px"), font_size=font_size, font_family=font_path))

    # Save the SVG file
    dwg.save()

In [ ]:
if __name__ == "__main__":
    font_path = src_dir + "handshape.TTF" # Replace with the path to your font file
    output_path = "output.svg"  # Replace with the desired output file path

    if os.path.isfile(font_path):
        generate_svg(font_path, output_path)
        print(f"SVG file generated: {output_path}")
    else:
        print("Invalid font file path.")


In [6]:
import fontTools.ttLib as tt
from paths import *
from PIL import Image, ImageDraw, ImageFont

In [5]:
def list_font_characters(font_path):
    font = tt.TTFont(font_path)
    cmap = font.getBestCmap()
    # characters = sorted(cmap.keys())
    characters = [chr(char_code) for char_code in sorted(cmap.keys())]
    return characters

font_path = src_dir + "handshape.TTF"
characters = list_font_characters(font_path)
print(characters)

[' ', '!', '"', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '‐']


In [29]:
font_size = 288
canvas_size = font_size * 2

In [31]:
for text in characters: 
    img = Image.new('RGBA', (canvas_size, canvas_size), color=(255, 0, 0, 0))
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, size=font_size)
    text_width, text_height = draw.textsize(text, font=font)
    x = (img.width - text_width) // 2
    y = (img.height - text_height) // 2
    draw.text((x, y), text, fill="black", font=font)
    output_file_path = src_dir + "hs/" + text + '.png'
    img.save(output_file_path, "PNG")

/tmp/ipykernel_1240378/3103861082.py:5: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_1240378/3103861082.py:5: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_1240378/3103861082.py:5: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_1240378/3103861082.py:5: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(text, font=font)
/tmp/ipykernel_1240378/3103861082.py:5: DeprecationWarning: textsize is deprecated and will be remov

In [ ]:
def extract_allkinds(self, filename): 
    # this one does not exclude polymorphemic signs, but remember, 
    # these signs are not accurate, 
    # we only take the first handshape as target. 
    entry = self.__search_by_name__(filename=filename)
    if entry.empty: 
        return GuideExtract()
    else: 
        # search result not empty
        only_1 = entry['ONLY_1'].iloc[0]
        if only_1 == 1: 
            side = entry['Side'].iloc[0]
            dh_1 = entry['DH_1'].iloc[0]
            oh_1 = entry['OH_1'].iloc[0]

            if side == 'D':
                right_hand_value = dh_1 if self.__value_acceptable__(dh_1) else None
                left_hand_value = oh_1 if self.__value_acceptable__(oh_1) else None
            elif side == 'S':
                left_hand_value = dh_1 if self.__value_acceptable__(dh_1) else None
                right_hand_value = oh_1 if self.__value_acceptable__(oh_1) else None
            return GuideExtract(
                monomorph=1, 
                dexter=right_hand_value, 
                sinister=left_hand_value
            )
        else: 
            # non-monomorphic
            return GuideExtract()